In [12]:
# Example 11-3. Partial sequence protocol implementation with __getitem__: 
# enough for item access, iteration and the in operator.
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos] # !!!
f = Foo()
print("f[0] = ", f[0])
print("f[1] = ", f[1])
print("f[2] = ", f[2])
for i in f: print(i)
print("20 in f : ", 20 in f)
print("15 in f : ", 15 in f)

f[0] =  0
f[1] =  10
f[2] =  20
0
10
20
20 in f :  True
15 in f :  False


In [22]:
# Example 11-5 - 6. Monkey patching FrenchDeck to make it mutable and compatible with random.shuffle
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    def __getitem__(self, position):
        return self._cards[position]
print("dir(FrenchDeck) : ", dir(FrenchDeck))
print()

from random import shuffle
def set_card(deck, position, card): # work however not set_card(self, key, value)
    deck._cards[position] = card
FrenchDeck.__setitem__ = set_card # !!!
print("dir(FrenchDeck) : ", dir(FrenchDeck))
print()
print("__setitem__" in dir(FrenchDeck))

dir(FrenchDeck) :  ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'ranks', 'suits']

dir(FrenchDeck) :  ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'ranks', 'suits']

True


In [28]:
# Example 11-8. frenchdeck2.py: FrenchDeck2, a subclass of collections.MutableSequence.
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    def __getitem__(self, position):
        return self._cards[position]
    def __setitem__(self, position, value):  # <1>
        self._cards[position] = value
    def __delitem__(self, position):  # <2>
        del self._cards[position]  # ??? why not: self._cards.del(position)
    def insert(self, position, value):  # <3>  ??? why not: __insert__
        self._cards.insert(position, value)

In [1]:
# Example 11-9. tombola.py: Tombola is an ABC with two abstract methods and two concrete methods
import abc

class Tombola(abc.ABC):  # <1>
    @abc.abstractmethod
    def load(self, iterable):  # <2>
        """Add items from an iterable."""
    @abc.abstractmethod
    def pick(self):  # <3>
        """Remove item at random, returning it.
        This method should raise `LookupError` when the instance is empty.
        """
    def loaded(self):  # <4>
        """Return `True` if there's at least 1 item, `False` otherwise."""
        return bool(self.inspect())  # <5>
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:  # <6>
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)  # <7>
        return tuple(sorted(items))

In [36]:
# Example 11-11. A fake Tombola doesn’t go undetected.
# from tombola import Tombola
class Fake(Tombola): #
    def pick(self):
        return 13
print("Fake : ", Fake)
f = Fake()

Fake :  <class '__main__.Fake'>


TypeError: Can't instantiate abstract class Fake with abstract methods load

In [12]:
# Example 11-12. bingo.py: BingoCage is a concrete subclass of Tombola.
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

import random
from tombola import Tombola

class BingoCage(Tombola):  # <1>
    def __init__(self, items):
        self._randomizer = random.SystemRandom()  # <2>
        self._items = []
        self.load(items)  # <3>
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)  # <4>
    def pick(self):  # <5>
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):  # <7>
        self.pick()

In [9]:
# Example 11-13. lotto.py: LotteryBlower is a concrete subclass 
# that overrides the in spect and loaded methods from Tombola.
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

import random
from tombola import Tombola

class LotteryBlower(Tombola):
    def __init__(self, iterable):
        self._balls = list(iterable)  # <1>
    def load(self, iterable):
        self._balls.extend(iterable)
    def pick(self):
        try:
            position = random.randrange(len(self._balls))  # <2>
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position)  # <3>
    def loaded(self):  # <4>
        return bool(self._balls)
    def inspect(self):  # <5>
        return tuple(sorted(self._balls))

In [17]:
# Example 11-14. tombolist.py: class TomboList is a virtual subclass of Tombola.
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

from random import randrange
from tombola import Tombola

@Tombola.register  # <1>
class TomboList(list):  # <2>
    def pick(self):
        if self:  # <3>
            position = randrange(len(self))
            return self.pop(position)  # <4>
        else:
            raise LookupError('pop from empty TomboList')
    load = list.extend  # <5> !!! bez def ???
    def loaded(self):
        return bool(self)  # <6>
    def inspect(self):
        return tuple(sorted(self))
# Tombola.register(TomboList)  # <7> using before Python 3.3 you must use standard call syntax.

print("issubclass(TomboList, Tombola) : ", issubclass(TomboList, Tombola))
t = TomboList(range(100))
print("isinstance(t, Tombola) : ", isinstance(t, Tombola))
print("TomboList.__mro__ : ", TomboList.__mro__)

issubclass(TomboList, Tombola) :  True
isinstance(t, Tombola) :  True
TomboList.__mro__ :  (<class '__main__.TomboList'>, <class 'list'>, <class 'object'>)


In [34]:
# Example 11-15. tombola_runner.py: test runner for Tombola subclasses.
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

import doctest
from tombola import Tombola

# modules to test
import bingo, lotto, tombolist, drum  # <1>

TEST_FILE = '/home/kwol/git/kw/jupyter/Fluent Python/files/tombola_tests.rst'
TEST_MSG = '{0:16} {1.attempted:2} tests, {1.failed:2} failed - {2}'


def main(argv):
    verbose = '-v' in argv
    real_subclasses = Tombola.__subclasses__()  # <2>
    virtual_subclasses = list(Tombola._abc_registry)  # <3>

    for cls in real_subclasses + virtual_subclasses:  # <4>
        test(cls, verbose)


def test(cls, verbose=False):

    res = doctest.testfile(
            TEST_FILE,
            module_relative=False,
            globs={'ConcreteTombola': cls},  # <5>
            verbose=verbose,
            optionflags=doctest.REPORT_ONLY_FIRST_FAILURE)
    tag = 'FAIL' if res.failed else 'OK'
    print(TEST_MSG.format(cls.__name__, res, tag))  # <6>


if __name__ == '__main__':
    import sys
    main(sys.argv)

BingoCage        24 tests,  0 failed - OK
LotteryBlower    24 tests,  0 failed - OK
BingoCage        24 tests,  0 failed - OK
LotteryBlower    24 tests,  0 failed - OK
BingoCage        24 tests,  0 failed - OK
TumblingDrum     24 tests,  0 failed - OK
TomboList        24 tests,  0 failed - OK
TomboList        24 tests,  0 failed - OK
TomboList        24 tests,  0 failed - OK


In [37]:
#  a class can be recognized as a virtual subclass of an ABC even without registration. 
class Struggle:
    def __len__(self): return 23
from collections import abc
print("isinstance(Struggle(), abc.Sized) : ", isinstance(Struggle(), abc.Sized))
print("issubclass(Struggle, abc.Sized) : ", issubclass(Struggle, abc.Sized))

isinstance(Struggle(), abc.Sized) :  True
issubclass(Struggle, abc.Sized) :  True


In [ ]:
# Example 11-17. Sized definition from the source code of Lib/_collections_abc.py
class Sized(metaclass=ABCMeta):
    __slots__ = ()
    @abstractmethod
    def __len__(self):
        return 0
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__): #
                return True #
    return NotImplemented #